## Kaggle data management api

In [7]:
# !pip install kaggle # now download the api token and store it to /home/ec2-user/.kaggle/kaggle.json
# !cd /home/ec2-user/SageMaker
# !mkdir /home/ec2-user/.kaggle/
# !mv /home/ec2-user/SageMaker/kaggle.json /home/ec2-user/.kaggle/kaggle.json
# !chmod 600 /home/ec2-user/.kaggle/kaggle.json # for privacy
!df -h --total  # get the harddisk info
!nvcc --version  # Get the cuda version


Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        3.8G   76K  3.8G   1% /dev
tmpfs           3.8G  4.0K  3.8G   1% /dev/shm
/dev/nvme0n1p1   94G   76G   19G  81% /
/dev/nvme1n1    985G  572G  373G  61% /home/ec2-user/SageMaker
total           1.1T  647G  399G  62% -
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [2]:
# !git clone https://github.com/RamsteinWR/COVIDNet.git
# !git clone https://github.com/ieee8023/covid-chestxray-dataset.git
# !git clone https://github.com/agchung/Figure1-COVID-chestxray-dataset.git
# !git clone https://github.com/agchung/Actualmed-COVID-chestxray-dataset.git

# !rm -rf /home/ec2-user/SageMaker/COVIDNet

In [8]:
# !pip install pydicom

In [3]:
%%time
import os
os.chdir("/home/ec2-user/SageMaker")

# Download the challenge data here 
# !kaggle competitions download -c rsna-pneumonia-detection-challenge
# !unzip -o -q /home/ec2-user/SageMaker/rsna-pneumonia-detection-challenge.zip -d /home/ec2-user/SageMaker/PneumoniaData  #-q for quitely no verbose
# !rm -rf /home/ec2-user/SageMaker/rsna-pneumonia-detection-challenge.zip

CPU times: user 97 µs, sys: 14 µs, total: 111 µs
Wall time: 117 µs


## Preprocessing the authenication management

In [5]:
%%time
import boto3
import botocore
from botocore.exceptions import ClientError
from tqdm import tqdm
import os 
import urllib.request
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

bucket = "covid19-pnemonia-dataset"
region_name="us-east-1"
aws_access_key_id = "jdhaiuhadi"
aws_secret_access_key = "hjafdvbujadh"

training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

def upload_to_s3(channel, file):
    s3 = boto3.resource('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)    
    data = open(file, "rb")
    key = channel + '/' + file
    print("Uploading file {} to s3://{}/{}".format(file, bucket, channel))
    s3.Bucket(bucket).put_object(Key=key, Body=data)

def upload_dir_to_s3(bucket, s3_folder, dir_to_upload):
    s3_client = boto3.client('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)
    print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_folder))
    # enumerate local files recursively
    for root, dirs, files in os.walk(dir_to_upload):
        for filename in tqdm(files):
            # construct the full local path
            local_path = os.path.join(root, filename)
            # construct the full Dropbox path
            relative_path = os.path.relpath(local_path, dir_to_upload)
            s3_path = os.path.join(s3_folder, relative_path).replace("\\", "/")
            try:
                s3_client.head_object(Bucket=bucket, Key=s3_path)
                print("Path found on S3! Deleting %s..." % s3_path)
                try:
                    s3_client.delete_object(Bucket=bucket, Key=s3_path)
                    try:
#                         print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_path)
                        s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                    except ClientError as e:
                        logging.error(e)
                except:
                    print("Unable to delete from s3 %s..." % s3_path)
            except:
                try:
                    s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                except ClientError as e:
                    logging.error(e)
    print("Upload completed successfully.")
    
def download_dir(s3_folder, local_path, bucket=""):
    """
    params:
    - s3_folder: pattern to match in s3
    - local_path: local_path path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    client = boto3.client('s3', region_name=region_name)
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket': bucket,
        'Prefix': s3_folder,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local_path, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    print("{} files found in {} directories. Downloading now...".format(len(keys), len(dirs)))
    for k in tqdm(keys):
        dest_pathname = os.path.join(local_path, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        try:
#             print("Downloading {}".format(dest_pathname))
            client.download_file(bucket, k, dest_pathname)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
    print("{} files downloaded successfully.".format(len(keys)))
    
# download_dir(s3_folder='Covid19Radiography', local_path='/home/ec2-user/SageMaker/data', bucket=bucket)
# !unzip -o -q /home/ec2-user/SageMaker/data/Covid19Radiography/COVID19RadiographyDatabase.zip -d /home/ec2-user/SageMaker/data/Covid19Radiography  #-q for quitely no verbose
# !rm -rf /home/ec2-user/SageMaker/data/Covid19Radiography/COVID19RadiographyDatabase.zip

# download_dir(s3_folder='COVIDNet-CXR_Large_Pretrained/', local_path='/home/ec2-user/SageMaker/data/COVIDNet-CXR-Large', bucket=bucket)


The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


CPU times: user 633 ms, sys: 243 ms, total: 875 ms
Wall time: 1.67 s


<a id='idg4c.1'></a>
## Dependencies
___
### import packages and check SageMaker version

In [5]:
import json
# import torch
import tarfile
import pickle
import matplotlib.pyplot as plt
# import torchvision as tv
import pathlib                          # Path management tool (standard library)
import subprocess                       # Runs shell commands via Python (standard library)
import sagemaker                        # SageMaker Python SDK
# from sagemaker.pytorch import PyTorch   # PyTorch Estimator for TensorFlow

### Data Preprocessing

In [9]:
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
import pydicom as dicom
from tqdm import tqdm
import cv2

ModuleNotFoundError: No module named 'pydicom'

In [ ]:
# set parameters here
savepath = '/home/ec2-user/SageMaker/data'
seed = 0
np.random.seed(seed) # Reset the seed so all runs are the same.
random.seed(seed)
MAXVAL = 255  # Range [0 255]

# path to covid-19 dataset from https://github.com/ieee8023/covid-chestxray-dataset
cohen_imgpath = '/home/ec2-user/SageMaker/covid-chestxray-dataset/images' 
cohen_csvpath = '/home/ec2-user/SageMaker/covid-chestxray-dataset/metadata.csv'

# path to covid-19 dataset from https://github.com/agchung/Figure1-COVID-chestxray-dataset
fig1_imgpath = '/home/ec2-user/SageMaker/Figure1-COVID-chestxray-dataset/images'
fig1_csvpath = '/home/ec2-user/SageMaker/Figure1-COVID-chestxray-dataset/metadata.csv'

# path to covid-19 dataset from https://github.com/agchung/Actualmed-COVID-chestxray-dataset
actmed_imgpath = '/home/ec2-user/SageMaker/Actualmed-COVID-chestxray-dataset/images'
actmed_csvpath = '/home/ec2-user/SageMaker/Actualmed-COVID-chestxray-dataset/metadata.csv'

# path to covid-19 dataset from https://www.kaggle.com/tawsifurrahman/covid19-radiography-database
sirm_imgpath = '/home/ec2-user/SageMaker/data/Covid19Radiography/COVID-19'
sirm_csvpath = '/home/ec2-user/SageMaker/data/Covid19Radiography/COVID-19.metadata.csv'

# path to https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
rsna_datapath = '/home/ec2-user/SageMaker/PneumoniaData'
# get all the normal from here
rsna_csvname = 'stage_2_detailed_class_info.csv' 
# get all the 1s from here since 1 indicate pneumonia
# found that images that aren't pneunmonia and also not normal are classified as 0s
rsna_csvname2 = 'stage_2_train_labels.csv' 
rsna_imgpath = 'train_dicoms'

# parameters for COVIDx dataset
train = []
test = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}

mapping = dict()
mapping['COVID-19'] = 'COVID-19'
mapping['SARS'] = 'pneumonia'
mapping['MERS'] = 'pneumonia'
mapping['Streptococcus'] = 'pneumonia'
mapping['Klebsiella'] = 'pneumonia'
mapping['Chlamydophila'] = 'pneumonia'
mapping['Legionella'] = 'pneumonia'
mapping['E.Coli'] = 'pneumonia'
mapping['Normal'] = 'normal'
mapping['Lung Opacity'] = 'pneumonia'
mapping['1'] = 'pneumonia'

# train/test split
split = 0.1

# to avoid duplicates
patient_imgpath = {}

In [ ]:
# adapted from https://github.com/mlmed/torchxrayvision/blob/master/torchxrayvision/datasets.py#L814
cohen_csv = pd.read_csv(cohen_csvpath, nrows=None)
#idx_pa = csv["view"] == "PA"  # Keep only the PA view
views = ["PA", "AP", "AP Supine", "AP semi erect", "AP erect"]
cohen_idx_keep = cohen_csv.view.isin(views)
cohen_csv = cohen_csv[cohen_idx_keep]

fig1_csv = pd.read_csv(fig1_csvpath, encoding='ISO-8859-1', nrows=None)
actmed_csv = pd.read_csv(actmed_csvpath, nrows=None)

sirm_csv = pd.read_csv(sirm_csvpath)

In [ ]:
# get non-COVID19 viral, bacteria, and COVID-19 infections from covid-chestxray-dataset, figure1 and actualmed
# stored as patient id, image filename and label
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
covid_ds = {'cohen': [], 'fig1': [], 'actmed': [], 'sirm': []}

for index, row in cohen_csv.iterrows():
    f = row['finding'].split('/')[-1] # take final finding in hierarchy, for the case of COVID-19, ARDS
    if f in mapping: # 
        count[mapping[f]] += 1
        entry = [str(row['patientid']), row['filename'], mapping[f], 'cohen']
        filename_label[mapping[f]].append(entry)
        if mapping[f] == 'COVID-19':
            covid_ds['cohen'].append(str(row['patientid']))
        
for index, row in fig1_csv.iterrows():
    if not str(row['finding']) == 'nan':
        f = row['finding'].split(',')[0] # take the first finding
        if f in mapping: # 
            count[mapping[f]] += 1
            if os.path.exists(os.path.join(fig1_imgpath, row['patientid'] + '.jpg')):
                entry = [row['patientid'], row['patientid'] + '.jpg', mapping[f], 'fig1']
            elif os.path.exists(os.path.join(fig1_imgpath, row['patientid'] + '.png')):
                entry = [row['patientid'], row['patientid'] + '.png', mapping[f], 'fig1']
            filename_label[mapping[f]].append(entry)
            if mapping[f] == 'COVID-19':
                covid_ds['fig1'].append(row['patientid'])

for index, row in actmed_csv.iterrows():
    if not str(row['finding']) == 'nan':
        f = row['finding'].split(',')[0]
        if f in mapping:
            count[mapping[f]] += 1
            entry = [row['patientid'], row['imagename'], mapping[f], 'actmed']
            filename_label[mapping[f]].append(entry)
            if mapping[f] == 'COVID-19':
                covid_ds['actmed'].append(row['patientid'])
    
sirm = set(sirm_csv['URL'])
cohen = set(cohen_csv['url'])
discard = ['100', '101', '102', '103', '104', '105', 
           '110', '111', '112', '113', '122', '123', 
           '124', '125', '126', '217']

for idx, row in sirm_csv.iterrows():
    patientid = row['FILE NAME']
    if row['URL'] not in cohen and patientid[patientid.find('(')+1:patientid.find(')')] not in discard:
        count[mapping['COVID-19']] += 1
        imagename = patientid + '.' + row['FORMAT'].lower()
        if not os.path.exists(os.path.join(sirm_imgpath, imagename)):
            imagename = patientid.split('(')[0] + ' ('+ patientid.split('(')[1] + '.' + row['FORMAT'].lower()
        entry = [patientid, imagename, mapping['COVID-19'], 'sirm']
        filename_label[mapping['COVID-19']].append(entry)
        covid_ds['sirm'].append(patientid)
    
print('Data distribution from covid datasets:')
print(count)

In [ ]:
# !cp -a -r /home/ec2-user/SageMaker/data/images /home/ec2-user/SageMaker/covid-chestxray-dataset/images

In [ ]:
# add covid-chestxray-dataset, figure1 and actualmed into COVIDx dataset
# since these datasets don't have test dataset, split into train/test by patientid
# for covid-chestxray-dataset:
# patient 8 is used as non-COVID19 viral test
# patient 31 is used as bacterial test
# patients 19, 20, 36, 42, 86 are used as COVID-19 viral test
# for figure 1:
# patients 24, 25, 27, 29, 30, 32, 33, 36, 37, 38

ds_imgpath = {'cohen': cohen_imgpath, 'fig1': fig1_imgpath, 'actmed': actmed_imgpath, 'sirm': sirm_imgpath}

for key in filename_label.keys():
    arr = np.array(filename_label[key])
    if arr.size == 0:
        continue
    # split by patients
    # num_diff_patients = len(np.unique(arr[:,0]))
    # num_test = max(1, round(split*num_diff_patients))
    # select num_test number of random patients
    # random.sample(list(arr[:,0]), num_test)
    if key == 'pneumonia':
        test_patients = ['8', '31']
    elif key == 'COVID-19':
        test_patients = ['19', '20', '36', '42', '86', 
                         '94', '97', '117', '132', 
                         '138', '144', '150', '163', '169', '174', '175', '179', '190', '191'
                         'COVID-00024', 'COVID-00025', 'COVID-00026', 'COVID-00027', 'COVID-00029',
                         'COVID-00030', 'COVID-00032', 'COVID-00033', 'COVID-00035', 'COVID-00036',
                         'COVID-00037', 'COVID-00038',
                         'ANON24', 'ANON45', 'ANON126', 'ANON106', 'ANON67',
                         'ANON153', 'ANON135', 'ANON44', 'ANON29', 'ANON201', 
                         'ANON191', 'ANON234', 'ANON110', 'ANON112', 'ANON73', 
                         'ANON220', 'ANON189', 'ANON30', 'ANON53', 'ANON46',
                         'ANON218', 'ANON240', 'ANON100', 'ANON237', 'ANON158',
                         'ANON174', 'ANON19', 'ANON195',
                         'COVID-19(119)', 'COVID-19(87)', 'COVID-19(70)', 'COVID-19(94)', 
                         'COVID-19(215)', 'COVID-19(77)', 'COVID-19(213)', 'COVID-19(81)', 
                         'COVID-19(216)', 'COVID-19(72)', 'COVID-19(106)', 'COVID-19(131)', 
                         'COVID-19(107)', 'COVID-19(116)', 'COVID-19(95)', 'COVID-19(214)', 
                         'COVID-19(129)']
    else: 
        test_patients = []
    print('Key: ', key)
    print('Test patients: ', test_patients)
    # go through all the patients
    for patient in arr:
        if patient[0] not in patient_imgpath:
            patient_imgpath[patient[0]] = [patient[1]]
        else:
            if patient[1] not in patient_imgpath[patient[0]]:
                patient_imgpath[patient[0]].append(patient[1])
            else:
                continue  # skip since image has already been written
        if patient[0] in test_patients:
            if patient[3] == 'sirm':
                image = cv2.imread(os.path.join(ds_imgpath[patient[3]], patient[1]))
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                patient[1] = patient[1].replace(' ', '')
                cv2.imwrite(os.path.join(savepath, 'test', patient[1]), gray)
            else:
                copyfile(os.path.join(ds_imgpath[patient[3]], patient[1]), os.path.join(savepath, 'test', patient[1]))
            test.append(patient)
            test_count[patient[2]] += 1
        else:
            if patient[3] == 'sirm':
                image = cv2.imread(os.path.join(ds_imgpath[patient[3]], patient[1]))
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                patient[1] = patient[1].replace(' ', '')
                cv2.imwrite(os.path.join(savepath, 'train', patient[1]), gray)
            else:
                copyfile(os.path.join(ds_imgpath[patient[3]], patient[1]), os.path.join(savepath, 'train', patient[1]))
            train.append(patient)
            train_count[patient[2]] += 1

print('test count: ', test_count)
print('train count: ', train_count)

In [ ]:
# add normal and rest of pneumonia cases from https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
csv_normal = pd.read_csv(os.path.join(rsna_datapath, rsna_csvname), nrows=None)
csv_pneu = pd.read_csv(os.path.join(rsna_datapath, rsna_csvname2), nrows=None)
patients = {'normal': [], 'pneumonia': []}

for index, row in csv_normal.iterrows():
    if row['class'] == 'Normal':
        patients['normal'].append(row['patientId'])

for index, row in csv_pneu.iterrows():
    if int(row['Target']) == 1:
        patients['pneumonia'].append(row['patientId'])

for key in patients.keys():
    arr = np.array(patients[key])
    if arr.size == 0:
        continue
    # split by patients 
    # num_diff_patients = len(np.unique(arr))
    # num_test = max(1, round(split*num_diff_patients))
    test_patients = np.load('rsna_test_patients_{}.npy'.format(key)) # random.sample(list(arr), num_test), download the .npy files from the repo.
    # np.save('rsna_test_patients_{}.npy'.format(key), np.array(test_patients))
    for patient in tqdm(arr):
        if patient not in patient_imgpath:
            patient_imgpath[patient] = [patient]
        else:
            continue  # skip since image has already been written
                
        ds = dicom.dcmread(os.path.join(rsna_datapath, rsna_imgpath, patient + '.dcm'))
        pixel_array_numpy = ds.pixel_array
        imgname = patient + '.png'
        if patient in test_patients:
            cv2.imwrite(os.path.join(savepath, 'test', imgname), pixel_array_numpy)
            test.append([patient, imgname, key, 'rsna'])
            test_count[key] += 1
        else:
            cv2.imwrite(os.path.join(savepath, 'train', imgname), pixel_array_numpy)
            train.append([patient, imgname, key, 'rsna'])
            train_count[key] += 1

print('test count: ', test_count)
print('train count: ', train_count)

In [ ]:
# final stats
print('Final stats')
print('Train count: ', train_count)
print('Test count: ', test_count)
print('Total length of train: ', len(train))
print('Total length of test: ', len(test))

In [ ]:
# export to train and test csv
# format as patientid, filename, label, separated by a space
train_file = open("/home/ec2-user/SageMaker/data/train_split.txt",'w') 
for sample in train:
    if len(sample) == 4:
        info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + ' ' + sample[3] + '\n'
    else:
        info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

test_file = open("/home/ec2-user/SageMaker/data/test_split.txt", 'w')
for sample in test:
    if len(sample) == 4:
        info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + ' ' + sample[3] + '\n'
    else:
        info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    test_file.write(info)

test_file.close()